#ElGamal encryption in Julia

###Karolina Szczygieł, Dawid Gajewy

##Auxiliary packages

In [ ]:
Pkg.add("Logging")

In [ ]:
using Logging
Logging.configure(level=ERROR)

##Packages

In [19]:
Pkg.clone("https://github.com/hwborchers/Numbers.jl.git")

INFO: Initializing package repository /home/juser/.julia/v0.4
INFO: Cloning METADATA from git://github.com/JuliaLang/METADATA.jl
INFO: Cloning Numbers from https://github.com/hwborchers/Numbers.jl.git
INFO: Computing changes...


In [13]:
using Numbers

##Types

In [6]:
immutable User
    name::UTF8String
    publicKey::Number # g^a
    privateKey::Number # a
end

In [7]:
immutable ElGamalBase
    p::Number # Prime number for Zp
    g::Number # Cyclic group generator
end

In [8]:
type KeyArray
    gk::Number
    M::Array{Number,1}
end

##Functions

###General

In [1]:
function generate_p(minRandPower::Number, maxRandPower::Number)
    return nextprime(rand(BigInt(2)^minRandPower:BigInt(2)^maxRandPower)) #1024,2048
    info("Wezmy ciało Zp dla p = ", nextprime)
end

generate_p (generic function with 1 method)

In [2]:
function return_ga(g::Number, a::Number, p::Number)
    ga =  powermod(g,a,p)
    info("Generowanie klucza \nElement g = ", g, " jest generatorem grupy Zp. \nOsoba wybiera a = ", a , " i oblicza \ng^a (mod p) = ", g ,"^", a , " (mod ", p, ") = ", ga)
    info("Osoba oglasza publicznie g^a = ", ga, ". Ponadto publicznie znane są również: p = ", p, " oraz g = ", g)
    return ga
end

return_ga (generic function with 1 method)

In [3]:
function string_to_intarray(message::UTF8String)
    A = Int[]
    for c in message
        push!(A,Int(c))
    end
    return A
end

string_to_intarray (generic function with 1 method)

In [4]:
function intarray_to_string(A::Array{Number,1})
    RA = Char[]
    for a in A
        push!(RA,Char(a))
    end
    return AbstractString(RA)
end

intarray_to_string (generic function with 1 method)

In [9]:
function create_user(base::ElGamalBase, name::AbstractString ,choosenNumber::Number)
    if choosenNumber > base.p-1
        choosenNumber = mod(choosenNumber,base.p)
        warn("Your choosen (private) number is higher than ElGamal prime. It will be converted to: ", choosenNumber)
    end
    return User(convert(UTF8String,name), return_ga(base.g,choosenNumber,base.p), choosenNumber)
end

create_user (generic function with 1 method)

###Encryption and decryption

In [10]:
function send_message(base::ElGamalBase, k::Number, addressee::User, message::AbstractString)
    if BigInt(k)>10000
        k = mod(k,10000)
        warn("For best performance, your key cannot be higher than 10 000. It was changed to: ", k)
    end
    # IA is an array of Integers converted from UTF8 string
    IA = string_to_intarray(convert(UTF8String,message))
    gk = powermod(BigInt(base.g),BigInt(k),BigInt(base.p))
    RA = Number[] #Result array (containt encrypted numbers)
    for a in IA
        m = powermod(BigInt(a)*(BigInt(addressee.publicKey)^BigInt(k)), BigInt(1), BigInt(base.p))
        push!(RA,BigInt(m))
    end
    info("Osoba A chce wyslac do osoby B wiadomosc P = ", message, ". Wybiera losowe k = ", k, " i oblicza \ng^k = ", base.g, "^", k, " (mod ", base.p, ") = ", gk, "\noraz \nP * (g^a)^k = ", message, "*", addressee.publicKey, " (mod ", base.p, ") = ", RA)
    info("Osoba A wysyła ", KeyArray(gk, RA), " do B")
    return KeyArray(gk, RA)
end

send_message (generic function with 1 method)

In [11]:
function read_message(base::ElGamalBase, recipient::User, CA::KeyArray)
    key = powermod(BigInt(CA.gk),BigInt(base.p)-BigInt(1)-BigInt(recipient.privateKey),BigInt(base.p))
    RA = Number[]
    for a in CA.M
        m = mod(BigInt(key)*BigInt(a),BigInt(base.p))
        push!(RA,BigInt(m))
    end
    info("Osoba B otrzymała (", (CA.gk),",", (CA.M), ") \nOblicza \ng^-ak = (g^k)^p-1-a = ", CA.gk, "^", BigInt(base.p)-BigInt(1)-BigInt(recipient.privateKey), " (mod ", base.p, ") = ", key, " \nUzyskuje wiadomosc \nP = ", key, "*", CA.M, " (mod ", base.p, ") = ", intarray_to_string(RA))
    return intarray_to_string(RA)
end

read_message (generic function with 1 method)

## Code

###Basic settings

####Base

In [14]:
ElGamal = ElGamalBase(generate_p(12,15),2) # creates p(prime number for Zp) and g(cyclic group generator)

ElGamalBase(24077,2)

In [ ]:
ElGamal = ElGamalBase(2357,2) #test

####Users

In [15]:
userA = create_user(ElGamal,"Dawid",172312313123123123112312312351) # creates user A with public and private key

User("Dawid",19231,9830)

INFO: Generowanie klucza 
Element g = 2 jest generatorem grupy Zp. 
Osoba wybiera a = 9830 i oblicza 
g^a (mod p) = 2^9830 (mod 24077) = 19231
INFO: Osoba oglasza publicznie g^a = 19231. Ponadto publicznie znane są również: p = 24077 oraz g = 2


In [ ]:
userA = create_user(ElGamal,"Karolina",2500)

###Encrypt mode

In [16]:
print("Write your message to encrypt: ")
message = convert(AbstractString,readline())

print("Write your key to encrypt above message (digits only): ")
key = try 
    parse(BigInt,readline())
    catch
        error("You have to write digits only!")
    end

CRYPTED = send_message(ElGamal, key, userA, message)
print("Your encrypted message is: ", CRYPTED)


Write your message to encrypt: STDIN> HaśHaś
Write your key to encrypt above message (digits only): STDIN> 12312


INFO: Osoba A chce wyslac do osoby B wiadomosc P = HaśHaś. Wybiera losowe k = 2312 i oblicza 
g^k = 2^2312 (mod 24077) = 17262
oraz 
P * (g^a)^k = HaśHaś*19231 (mod 24077) = Number[11596,16960,22446,11596,16960,22446]
INFO: Osoba A wysyła KeyArray(17262,Number[11596,16960,22446,11596,16960,22446]) do B


Your encrypted message is: KeyArray(17262,Number[11596,16960,22446,11596,16960,22446])

###Decrypt mode

In [18]:
DECRYPTED = read_message(ElGamal,userA,CRYPTED)

INFO: Osoba B otrzymała (17262,Number[11596,16960,22446,11596,16960,22446]) 
Oblicza 
g^-ak = (g^k)^p-1-a = 17262^14246 (mod 24077) = 8488 
Uzyskuje wiadomosc 
P = 8488*Number[11596,16960,22446,11596,16960,22446] (mod 24077) = HaśHaś


"HaśHaś"